In [518]:
import numpy as np
import cv2 as cv

In [527]:
DEBUG = 0
win = 100

In [520]:
def win_init():
    cv.namedWindow('img', cv.WINDOW_NORMAL)
    cv.resizeWindow('img', 400, 400)
    cv.namedWindow('th', cv.WINDOW_NORMAL) 
    cv.resizeWindow('th', 400, 400)
    cv.namedWindow('draw', cv.WINDOW_NORMAL)
    cv.resizeWindow('draw', 400, 400)
def lst_square(points):
    # inv(x.T * x) * x.T * Y
    dx = np.ptp(points[1])
    dy = np.ptp(points[0])
    # print(dx, dy)
    if dx > dy:
        x = np.ones((points[1].size, 2))
        x[:, 0] = points[1]
        # y = 99 - points[0].reshape(points[0].size, 1)
        y = points[0].reshape(points[0].size, 1)
        return np.linalg.inv(np.dot(x.T, x)).dot(x.T).dot(y)
    else:
        x = np.ones((points[0].size, 2))
        x[:, 0] = points[0]
        y = points[1].reshape(points[1].size, 1)
        temp = np.linalg.inv(np.dot(x.T, x)).dot(x.T).dot(y)
        # print(temp)
        return [1 / temp[0], - temp[1] / temp[0]]
        # return [1 / temp[0], - temp[1] / temp[0]]
    
def draw(img, theta):
    x = np.linspace(0, 99, 100).astype(np.int64)
    y = np.floor(theta[0] * x + theta[1]).astype(np.int64)
    img[99 - y[np.where((y > 0) &(y < 100))], x[np.where((y > 0) &(y < 100))]] = 150 
    
def threshold(img):
    hist = cv.calcHist([img],[0],None,[256],[0,256])
    cnt = 0
    th = 0
    for k in range(256):
        cnt = cnt + hist[255 - k]
        if cnt > 20:
            th = 255 - k - 1
            break
    ret, thImg = cv.threshold(img,th,255,cv.THRESH_BINARY)
    return thImg

def collinear_rate(points, theta):
    div = np.linalg.norm([theta[0], 1])
    D = np.abs(theta[0] * points[1] - points[0] + theta[1])
    
    ret = np.mean(D / div)
    return ret
    

In [543]:
def solution(image):
    row, col = image.shape
    blurImg = cv.GaussianBlur(image, (3, 3), 0)
    Theta = list([])
    
    for i in range(row // win):
        for j in range(col // win):
            drawImg = np.zeros((100, 100))
            window = blurImg[i * win : (i + 1) * win, j * win : (j + 1) * win]
            mean = np.mean(window)
            std = np.std(window)
            if mean > 180:
                continue
            thImg = threshold(window)
            points = np.array(np.where(thImg == 255))
            # 1. del outlier
            mean_x = int(np.mean(points[1]))
            mean_y = int(np.mean(points[0]))
            thImg[mean_y, mean_x] = 100
            dx = np.abs(points[1] - mean_x)
            dy = np.abs(points[0] - mean_y)
            points = points[:, np.where(dx + dy < 50)].reshape(2, -1)
            if points[0].size < 2:
                continue
            mean_x = int(np.mean(points[1]))
            mean_y = int(np.mean(points[0]))
            thImg[mean_y, mean_x] = 100
            dx = np.abs(points[1] - mean_x)
            dy = np.abs(points[0] - mean_y)
            points = points[:, np.where(dx + dy < 30)]
            if points[0].size < 2:
                continue
            # 2. Check for discreteness
            std_x = np.std(points[1])
            std_y = np.std(points[0])
        
            if std_x + std_y > 30 or std_x * std_y > 50:
                continue
            # print(points)
            # print(std_x + std_y, '   ', std_x * std_y)
            
            # 3. least square fit
            if points[0].size < 10:
                continue
            # print(points.shape)
            points[0] = 99 - points[0]
            theta = lst_square(points)
            
            # 4. Check for linearity
            nonlinear = collinear_rate(points, theta)
            if nonlinear > 0.8:
                continue
            # print(nonlinear)
            
            if DEBUG:
                draw(drawImg, theta)
                cv.imshow('img', window)
                cv.imshow('th', thImg)
                cv.imshow('draw', drawImg)
                print('a = ', theta[0], 'b = ', theta[1])
                print('---------------------------------------')
                key = cv.waitKey(0)
            else:
                key = cv.waitKey(1)
                
            # 5. Add theta to Theta
            theta = np.arctan(theta[0]) * 180 / np.pi
            Theta.append(theta)
            # print(theta[0])
                        
            
            if key == 27:
                cv.destroyAllWindows()
                return
    # 7. del outlier
    Theta.sort()
    std = np.std(Theta)
    while std > 0.6:
        a = Theta[1:]
        b = Theta[:-1]
        Theta = a if np.std(a) < np.std(b) else b
        std = np.std(Theta)
    if len(Theta) > 0:
        print('theta:', np.mean(Theta))
        if DEBUG:
            print('Theta:', Theta)
            
    cv.destroyAllWindows()

In [567]:
DEBUG = 0
src = cv.imread('0.png', 0)
if DEBUG:
    win_init()
solution(src)
cv.destroyAllWindows()

theta: -0.206566296976
